# Using GRU Model for testing Ambig QA

In [1]:
import numpy as np
import pandas as pd 
import torch
import torchtext
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from torchtext.datasets import Multi30k
from torchtext.data import Field, BucketIterator

import spacy
import numpy as np

import random
import math
import time

In [2]:
!ls

Break-dataset.zip  sample_data


In [3]:
!unzip /content/Break-dataset.zip

Archive:  /content/Break-dataset.zip
   creating: Break-dataset/logical-forms/
  inflating: Break-dataset/logical-forms/dev.csv  
  inflating: Break-dataset/logical-forms/test.csv  
  inflating: Break-dataset/logical-forms/train.csv  
   creating: Break-dataset/QDMR/
  inflating: Break-dataset/QDMR/dev.csv  
  inflating: Break-dataset/QDMR/dev_lexicon_tokens.json  
  inflating: Break-dataset/QDMR/test.csv  
  inflating: Break-dataset/QDMR/test_lexicon_tokens.json  
  inflating: Break-dataset/QDMR/train.csv  
  inflating: Break-dataset/QDMR/train_lexicon_tokens.json  
   creating: Break-dataset/QDMR-high-level/
  inflating: Break-dataset/QDMR-high-level/dev.csv  
  inflating: Break-dataset/QDMR-high-level/dev_lexicon_tokens.json  
  inflating: Break-dataset/QDMR-high-level/test.csv  
  inflating: Break-dataset/QDMR-high-level/test_lexicon_tokens.json  
  inflating: Break-dataset/QDMR-high-level/train.csv  
  inflating: Break-dataset/QDMR-high-level/train_lexicon_tokens.json  


In [4]:
df = pd.read_csv('/content/Break-dataset/QDMR/train.csv')


In [25]:
df.head(5)

,question_id,question_text,decomposition,operators,split
0,ACADEMIC_train_0,return me the homepage of PVLDB .,return homepages ;return #1 of PVLDB,"['select', 'filter']",train
1,ACADEMIC_train_1,"return me the homepage of "" H. V. Jagadish "" .",return homepages ;return #1 of H. V. Jagadish,"['select', 'filter']",train
2,ACADEMIC_train_10,"return me the number of references of "" Making...",return references ;return #1 of Making databa...,"['select', 'filter', 'aggregate']",train
3,ACADEMIC_train_100,return me the number of papers in VLDB confere...,return papers ;return #1 in VLDB conference ;r...,"['select', 'filter', 'filter', 'aggregate']",train
4,ACADEMIC_train_101,return me the number of authors who have paper...,return papers ;return #1 containing keyword Re...,"['select', 'filter', 'project', 'aggregate']",train


In [26]:
df.columns

Index(['question_id', 'question_text', 'decomposition', 'operators', 'split'], dtype='object')

In [27]:
df.columns

Index(['question_id', 'question_text', 'decomposition', 'operators', 'split'], dtype='object')

In [28]:
data = df.rename(columns={'question_text':'Question','decomposition':'Answer'})

data.to_csv('train_data.csv',index=False)
#column_names = ['id','question','nq_answer','nq_doc_title']
#data = df.reindex(columns= column_names)

In [29]:
df1 = data[['Question','Answer']]

In [30]:
#df1 = pd.read_csv('train_data.csv')
df1.head(5)

,Question,Answer
0,return me the homepage of PVLDB .,return homepages ;return #1 of PVLDB
1,"return me the homepage of "" H. V. Jagadish "" .",return homepages ;return #1 of H. V. Jagadish
2,"return me the number of references of "" Making...",return references ;return #1 of Making databa...
3,return me the number of papers in VLDB confere...,return papers ;return #1 in VLDB conference ;r...
4,return me the number of authors who have paper...,return papers ;return #1 containing keyword Re...


In [31]:
#df1["Answer"] = df1["Answer"].str.split(";") 
df1

,Question,Answer
0,return me the homepage of PVLDB .,return homepages ;return #1 of PVLDB
1,"return me the homepage of "" H. V. Jagadish "" .",return homepages ;return #1 of H. V. Jagadish
2,"return me the number of references of "" Making...",return references ;return #1 of Making databa...
3,return me the number of papers in VLDB confere...,return papers ;return #1 in VLDB conference ;r...
4,return me the number of authors who have paper...,return papers ;return #1 containing keyword Re...
...,...,...
44316,What are the average enrollment size of the un...,return universities ;return #1 founded before ...
44317,Return the average enrollment of universities ...,return universities ;return #1 founded before ...
44318,Show the enrollment and primary_conference of ...,return colleges ;return the oldest #1 ;return...
44319,What are the enrollment and primary conference...,return universities ;return #1 founded earlies...


In [32]:
data
#data.dropna(how='any')

,question_id,Question,Answer,operators,split
0,ACADEMIC_train_0,return me the homepage of PVLDB .,return homepages ;return #1 of PVLDB,"['select', 'filter']",train
1,ACADEMIC_train_1,"return me the homepage of "" H. V. Jagadish "" .",return homepages ;return #1 of H. V. Jagadish,"['select', 'filter']",train
2,ACADEMIC_train_10,"return me the number of references of "" Making...",return references ;return #1 of Making databa...,"['select', 'filter', 'aggregate']",train
3,ACADEMIC_train_100,return me the number of papers in VLDB confere...,return papers ;return #1 in VLDB conference ;r...,"['select', 'filter', 'filter', 'aggregate']",train
4,ACADEMIC_train_101,return me the number of authors who have paper...,return papers ;return #1 containing keyword Re...,"['select', 'filter', 'project', 'aggregate']",train
...,...,...,...,...,...
44316,SPIDER_train_995,What are the average enrollment size of the un...,return universities ;return #1 founded before ...,"['select', 'filter', 'project', 'aggregate']",train
44317,SPIDER_train_996,Return the average enrollment of universities ...,return universities ;return #1 founded before ...,"['select', 'filter', 'project', 'aggregate']",train
44318,SPIDER_train_997,Show the enrollment and primary_conference of ...,return colleges ;return the oldest #1 ;return...,"['select', 'project', 'project', 'project', 'u...",train
44319,SPIDER_train_998,What are the enrollment and primary conference...,return universities ;return #1 founded earlies...,"['select', 'filter', 'project', 'project', 'un...",train


In [33]:
TEXT = torchtext.data.Field(
    tokenize='spacy',
    lower=True,
    init_token='<sos>', 
    eos_token='<eos>'
)

fields = [('question', TEXT), ('nq_answer', TEXT)]

In [34]:
# Create dataset
start = time.time()
dataset = torchtext.data.TabularDataset(
    path='train_data.csv',
    format='CSV',
    fields=fields,
    skip_header=True
)
end = time.time()
print(f'Duration: {end - start}')

Duration: 9.816263675689697


In [35]:
# Train/val split
train_data, valid_data = dataset.split(split_ratio=[0.85, 0.15])
print(len(train_data), len(valid_data))

37673 6648


In [36]:
vars(train_data.examples[0])

{'nq_answer': ['how',
  'far',
  'was',
  'the',
  'first',
  'scoring',
  'pass',
  'that',
  'the',
  'steeler',
  "'s",
  'quarterback',
  'made',
  '?'],
 'question': ['drop_train_nfl_2471_1a2fac39',
  '-',
  '71ff-4dec-8e29',
  '-',
  '4032a2cb6291']}

In [37]:
MAX_VOCAB_SIZE = 10_000

TEXT.build_vocab(train_data, max_size=MAX_VOCAB_SIZE)

print(f'Size of vocab: {len(TEXT.vocab)}')


Size of vocab: 10004


In [38]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [39]:
BATCH_SIZE = 128

train_iterator, valid_iterator = torchtext.data.BucketIterator.splits(
    (train_data, valid_data),
    batch_size = BATCH_SIZE,
    sort_key=lambda x: len(x.question),
    device = device
)

In [40]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout):
        super().__init__()
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.GRU(emb_dim, enc_hid_dim, bidirectional = True)
        
        self.fc = nn.Linear(enc_hid_dim * 2, dec_hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, hidden = self.rnn(embedded)
                
        #outputs = [src len, batch size, hid dim * num directions]
        #hidden = [n layers * num directions, batch size, hid dim]
        
        #hidden is stacked [forward_1, backward_1, forward_2, backward_2, ...]
        #outputs are always from the last layer
        
        #hidden [-2, :, : ] is the last of the forwards RNN 
        #hidden [-1, :, : ] is the last of the backwards RNN
        
        #initial decoder hidden is final hidden state of the forwards and backwards 
        #  encoder RNNs fed through a linear layer
        hidden = torch.tanh(self.fc(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1)))
        
        #outputs = [src len, batch size, enc hid dim * 2]
        #hidden = [batch size, dec hid dim]
        
        return outputs, hidden


In [41]:
# Attention

class Attention(nn.Module):
    def __init__(self, enc_hid_dim, dec_hid_dim):
        super().__init__()
        
        self.attn = nn.Linear((enc_hid_dim * 2) + dec_hid_dim, dec_hid_dim)
        self.v = nn.Linear(dec_hid_dim, 1, bias = False)
        
    def forward(self, hidden, encoder_outputs):
        
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        batch_size = encoder_outputs.shape[1]
        src_len = encoder_outputs.shape[0]
        
        #repeat decoder hidden state src_len times
        hidden = hidden.unsqueeze(1).repeat(1, src_len, 1)
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #hidden = [batch size, src len, dec hid dim]
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim = 2))) 
        
        #energy = [batch size, src len, dec hid dim]

        attention = self.v(energy).squeeze(2)
        
        #attention= [batch size, src len]
        
        return F.softmax(attention, dim=1)



In [42]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, enc_hid_dim, dec_hid_dim, dropout, attention):
        super().__init__()

        self.output_dim = output_dim
        self.attention = attention
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.GRU((enc_hid_dim * 2) + emb_dim, dec_hid_dim)
        
        self.fc_out = nn.Linear((enc_hid_dim * 2) + dec_hid_dim + emb_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, encoder_outputs):
             
        #input = [batch size]
        #hidden = [batch size, dec hid dim]
        #encoder_outputs = [src len, batch size, enc hid dim * 2]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
        
        a = self.attention(hidden, encoder_outputs)
                
        #a = [batch size, src len]
        
        a = a.unsqueeze(1)
        
        #a = [batch size, 1, src len]
        
        encoder_outputs = encoder_outputs.permute(1, 0, 2)
        
        #encoder_outputs = [batch size, src len, enc hid dim * 2]
        
        weighted = torch.bmm(a, encoder_outputs)
        
        #weighted = [batch size, 1, enc hid dim * 2]
        
        weighted = weighted.permute(1, 0, 2)
        
        #weighted = [1, batch size, enc hid dim * 2]
        
        rnn_input = torch.cat((embedded, weighted), dim = 2)
        
        #rnn_input = [1, batch size, (enc hid dim * 2) + emb dim]
            
        output, hidden = self.rnn(rnn_input, hidden.unsqueeze(0))
        
        #output = [seq len, batch size, dec hid dim * n directions]
        #hidden = [n layers * n directions, batch size, dec hid dim]
        
        #seq len, n layers and n directions will always be 1 in this decoder, therefore:
        #output = [1, batch size, dec hid dim]
        #hidden = [1, batch size, dec hid dim]
        #this also means that output == hidden
        assert (output == hidden).all()
        
        embedded = embedded.squeeze(0)
        output = output.squeeze(0)
        weighted = weighted.squeeze(0)
        
        prediction = self.fc_out(torch.cat((output, weighted, embedded), dim = 1))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden.squeeze(0)

In [43]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
    def forward(self, src, trg, teacher_forcing_ratio = 0.5):
        
        #src = [src len, batch size]
        #trg = [trg len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use teacher forcing 75% of the time
        
        batch_size = src.shape[1]
        trg_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        
        #tensor to store decoder outputs
        outputs = torch.zeros(trg_len, batch_size, trg_vocab_size).to(self.device)
        
        #encoder_outputs is all hidden states of the input sequence, back and forwards
        #hidden is the final forward and backward hidden states, passed through a linear layer
        encoder_outputs, hidden = self.encoder(src)
                
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, trg_len):
            
            #insert input token embedding, previous hidden state and all encoder hidden states
            #receive output tensor (predictions) and new hidden state
            output, hidden = self.decoder(input, hidden, encoder_outputs)
            
            #place predictions in a tensor holding predictions for each token
            outputs[t] = output
            
            #decide if we are going to use teacher forcing or not
            teacher_force = random.random() < teacher_forcing_ratio
            
            #get the highest predicted token from our predictions
            top1 = output.argmax(1) 
            
            #if teacher forcing, use actual next token as next input
            #if not, use predicted token
            input = trg[t] if teacher_force else top1

        return outputs

In [44]:
INPUT_DIM = len(TEXT.vocab)
OUTPUT_DIM = len(TEXT.vocab)
ENC_EMB_DIM = 128
DEC_EMB_DIM = 128
ENC_HID_DIM = 512
DEC_HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

attn = Attention(ENC_HID_DIM, DEC_HID_DIM)
#enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, ENC_DROPOUT)
#dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, DEC_HID_DIM, DEC_DROPOUT)
enc = Encoder(INPUT_DIM, ENC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, ENC_HID_DIM, DEC_HID_DIM, DEC_DROPOUT, attn)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

In [45]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(10004, 128)
    (rnn): GRU(128, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
  (decoder): Decoder(
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (embedding): Embedding(10004, 128)
    (rnn): GRU(1152, 512)
    (fc_out): Linear(in_features=1664, out_features=10004, bias=True)
    (dropout): Dropout(p=0.5, inplace=False)
  )
)

In [46]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 25,061,140 trainable parameters


In [47]:
optimizer = optim.Adam(model.parameters())

In [48]:


TRG_PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

criterion = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)



In [49]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.question
        trg = batch.nq_answer
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg len, batch size]
        #output = [trg len, batch size, output dim]
        
        output_dim = output.shape[-1]
        
        output = output[1:].view(-1, output_dim)
        trg = trg[1:].view(-1)
        
        #trg = [(trg len - 1) * batch size]
        #output = [(trg len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [50]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.question
            trg = batch.nq_answer

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg len, batch size]
            #output = [trg len, batch size, output dim]

            output_dim = output.shape[-1]
            
            output = output[1:].view(-1, output_dim)
            trg = trg[1:].view(-1)

            #trg = [(trg len - 1) * batch size]
            #output = [(trg len - 1) * batch size, output dim]

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

In [51]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [52]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

Epoch: 01 | Time: 1m 48s
	Train Loss: 5.107 | Train PPL: 165.092
	 Val. Loss: 5.575 |  Val. PPL: 263.755
Epoch: 02 | Time: 1m 54s
	Train Loss: 4.328 | Train PPL:  75.807
	 Val. Loss: 5.496 |  Val. PPL: 243.760
Epoch: 03 | Time: 1m 55s
	Train Loss: 4.116 | Train PPL:  61.287
	 Val. Loss: 5.359 |  Val. PPL: 212.473
Epoch: 04 | Time: 1m 54s
	Train Loss: 3.985 | Train PPL:  53.766
	 Val. Loss: 5.473 |  Val. PPL: 238.120
Epoch: 05 | Time: 1m 55s
	Train Loss: 3.843 | Train PPL:  46.677
	 Val. Loss: 5.547 |  Val. PPL: 256.513
Epoch: 06 | Time: 1m 55s
	Train Loss: 3.757 | Train PPL:  42.835
	 Val. Loss: 5.526 |  Val. PPL: 251.214
Epoch: 07 | Time: 1m 54s
	Train Loss: 3.720 | Train PPL:  41.272
	 Val. Loss: 5.538 |  Val. PPL: 254.148
Epoch: 08 | Time: 1m 54s
	Train Loss: 3.661 | Train PPL:  38.897
	 Val. Loss: 5.440 |  Val. PPL: 230.528
Epoch: 09 | Time: 1m 54s
	Train Loss: 3.593 | Train PPL:  36.328
	 Val. Loss: 5.581 |  Val. PPL: 265.365
Epoch: 10 | Time: 1m 54s
	Train Loss: 3.551 | Train PPL